# Init

## Load

In [8]:
%cd ..
%pwd

/Users/wliao0504/code/clif


/Users/wliao0504/code/clif/pyCLIF/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/Users/wliao0504/code/clif'

In [ ]:
from clifpy import MedicationAdminContinuous 

mac = MedicationAdminContinuous.from_file(
    data_directory = '/Users/wliao0504/code/clif/ucmc-clif-data',
    filetype = 'parquet'
)

In [49]:
mac_df = mac.df

In [ ]:
import pandas as pd
import duckdb

def find_mar_duplicates(meds_df: pd.DataFrame, ignore_zero_dose: bool = False) -> pd.DataFrame:
    print(f"Checking for duplicates to enable pivoting to wide format")

    med_category_column_name = 'med_category'
    # first drop exact-value, non-conflicting duplicates
    meds_df.drop_duplicates(subset=['hospitalization_id', 'admin_dttm', med_category_column_name, 'med_dose'], inplace=True)
    mask = meds_df.duplicated(subset=['hospitalization_id', 'admin_dttm', med_category_column_name], keep=False)
    dups = meds_df[mask]
    q = """
    SELECT *
        , combo: STRING_AGG(mar_action_name, '; ' ORDER BY mar_action_name) OVER (PARTITION BY hospitalization_id, med_category, admin_dttm)
    FROM dups
    """
    dups_w_combo = duckdb.sql(q).df()
    if not ignore_zero_dose:
        print(f"Found {len(dups_w_combo)} duplicate entries")
        return dups_w_combo.sort_values(by=['hospitalization_id', 'med_category', 'admin_dttm', 'mar_action_name'])
    non_zero_dups = dups_w_combo.query('med_dose > 0')
    mask = non_zero_dups.duplicated(subset=['hospitalization_id', 'admin_dttm', med_category_column_name], keep=False)
    non_zero_dups = non_zero_dups[mask]
    print(f"Found {len(non_zero_dups)} non-zero-dose duplicate entries")
    return non_zero_dups.sort_values(by=['hospitalization_id', 'med_category', 'admin_dttm', 'mar_action_name'])
    
def generate_combo_counts(dups_df: pd.DataFrame) -> pd.DataFrame:
    q = """
    SELECT 
        combo, COUNT(*) as count
    FROM dups_df
    GROUP BY combo
    ORDER BY count DESC
    """
    combo_counts = duckdb.sql(q).df()
    
    q = """
    SELECT *
    FROM dups_df
    WHERE combo in (
        SELECT combo FROM combo_counts 
        WHERE contains(combo, 'Given; Rate Change') OR contains(combo, 'Given; New Bag')
        --WHERE count > 100 AND NOT contains(combo, 'Canceled') AND NOT contains(combo, 'Verif')
    )
    """
    top_dups = duckdb.sql(q).df()
    
    return combo_counts, top_dups

dups = find_mar_duplicates(mac.df, ignore_zero_dose = False)#.head(10)
combo_counts, top_dups = generate_combo_counts(dups)

Checking for duplicates to enable pivoting to wide format
Found 10312 duplicate entries
